In [1]:
from pyspark.sql import SQLContext
import pandas as pd
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

In [2]:
sc = SparkContext()
sqlContext = SQLContext(sc)
spark = SparkSession(sc)

In [3]:
def run_sql(statement):
    try:
        result = sqlContext.sql(statement)
    except Exception as e:
        print(e.desc, '\n', e.stackTrace)
        return
    return result

In [4]:
run_sql('create database review_matrix')
dbs = run_sql('show databases')

In [5]:
bookings_df = (spark.read.format("csv") 
                    .option("inferSchema", "true") 
                    .option("header", "true") 
                    .option("sep", ",") 
                    .load("data/review_df.csv"))

In [6]:
bookings_df.dtypes

[('stars', 'string'),
 ('text', 'string'),
 ('useful', 'string'),
 ('nb_days', 'string'),
 ('log_useful', 'string')]

In [7]:
bookings_df.printSchema()

root
 |-- stars: string (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: string (nullable = true)
 |-- nb_days: string (nullable = true)
 |-- log_useful: string (nullable = true)

